# 모델 1 - 기능 1, 2, 3에 대해 처리 가능한 모델1 작성 (2)
#### ver 3.3
#### 최종 수정일 : 20-05-25
#### 이화여대 컴퓨터공학전공 졸업 프로젝트
#### 위잉위잉(척척고졸)-권지현

## LSTM 모델

* * *
## 2. 모델링
### 2.1 데이터 구성

> [변경사항]

* * *
> [기존과 동일]
- model_1.npz 불러오기

In [58]:
import glob
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [59]:
# 알림, 차량 엔진, 차량 경적, 지하철 트리거 순
sound_data = np.load('model_1.npz')
X_train = sound_data['X']
y_train = sound_data['y']
X_train.shape, y_train.shape

((3497, 193), (3497, 1))

In [60]:
X_train.shape, y_train.shape

((3497, 193), (3497, 1))

* * *
## 2. 모델링
### 2.2 모델 학습

> [변경사항]
- lstm 사용
- 이전 모델의 경우 파라미터 조정에 초점을 두었으나 ver 3.5에서는 layer 구성에 초점을 두어 진행
* * *
> [기존과 동일]


In [61]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import os

In [62]:
from keras import models
from keras import layers
from keras.layers import *
from keras import optimizers

In [63]:
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers import Dense
import keras.backend as K
from keras.callbacks import EarlyStopping

K.clear_session()
model = Sequential() # Sequeatial Model
model.add(LSTM(20, input_shape=(193, 1))) # (timestep, feature)
model.add(Dense(1)) # output = 1
model.compile(loss='mean_squared_error', optimizer='adam')

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 20)                1760      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 21        
Total params: 1,781
Trainable params: 1,781
Non-trainable params: 0
_________________________________________________________________


In [64]:
#X_train = X_train.values
X_train = X_train.reshape(X_train.shape[0], 193, 1)

In [65]:
early_stop = EarlyStopping(monitor='loss', patience=1, verbose=1)

model.fit(X_train, y_train, epochs=100,
          batch_size=30, verbose=1, callbacks=[early_stop])

Epoch 1/100
3497/3497 [==============================] - 7s 2ms/step - loss: 1.5702
Epoch 2/100
3497/3497 [==============================] - 7s 2ms/step - loss: 1.2128
Epoch 3/100
3497/3497 [==============================] - 8s 2ms/step - loss: 1.1071
Epoch 4/100
3497/3497 [==============================] - 8s 2ms/step - loss: 1.0286
Epoch 5/100
3497/3497 [==============================] - 8s 2ms/step - loss: 0.9506
Epoch 6/100
3497/3497 [==============================] - 8s 2ms/step - loss: 0.9092
Epoch 7/100
3497/3497 [==============================] - 8s 2ms/step - loss: 0.8712
Epoch 8/100
3497/3497 [==============================] - 8s 2ms/step - loss: 0.8550
Epoch 9/100
3497/3497 [==============================] - 8s 2ms/step - loss: 0.8209
Epoch 10/100
3497/3497 [==============================] - 8s 2ms/step - loss: 0.8035
Epoch 11/100
3497/3497 [==============================] - 8s 2ms/step - loss: 0.7542
Epoch 12/100
3497/3497 [==============================] - 8s 2ms/step - lo

* * *
## 2. 모델링
### 2.3 모델 저장

> [변경사항]
- pkl, json, pb, tflite로 저장
* * *
> [기존과 동일]


In [66]:
# 모델 pkl로 저장하기
import joblib
joblib.dump(model, 'model/pkl/model_1.pkl')

['model/pkl/model_1.pkl']

In [67]:
# 모델 json으로 저장하기
model_1 = model.to_json()
# model = model_from_json(json_string)

In [69]:
# 모델 h5로 저장하기
from keras.models import load_model
model.save('model/h5/model_1')
model.save('model/h5/model_1.h5')

In [70]:
# 모델 pb로 저장하기
model = keras.models.load_model('model/h5/model_1', compile=False)
model.save('model/pb/',save_format=tf)

INFO:tensorflow:Assets written to: model/pb/assets


In [72]:
#모델 tflite 로 저장하기
saved_model_dir='model/pb/'
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.target_spec.supported_ops=[tf.lite.OpsSet.TFLITE_BUILTINS,
                                     tf.lite.OpsSet.SELECT_TF_OPS]
tfilte_mode=converter.convert()
open('model/tflite/model_1.tflite','wb').write(ftlite_model)

ConverterError: See console for info.
2020-05-26 22:18:59.778839: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'cudart64_101.dll'; dlerror: cudart64_101.dll not found
2020-05-26 22:18:59.779224: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2020-05-26 22:19:03.763709: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2
2020-05-26 22:19:03.765597: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library nvcuda.dll
2020-05-26 22:19:03.794741: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce 940MX computeCapability: 5.0
coreClock: 1.2415GHz coreCount: 3 deviceMemorySize: 2.00GiB deviceMemoryBandwidth: 14.92GiB/s
2020-05-26 22:19:03.796081: W tensorfl